In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 31.3 MB/s 
     |████████████████████████████████| 163 kB 72.8 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [4]:
posts_30K_each = pd.read_csv('/content/drive/MyDrive/TCC II/all_30k_processed_posts.csv')

In [5]:
posts_30K_each

,Unnamed: 0,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message,author,label
0,17518,2020-06-08T16:20:29+0000,167637636622585_3560581307328184,exministro juiz sergio moro detonou conducao g...,o ex-ministro e juiz sergio moro detonou a con...,291,shared_story,https://external.fplu2-1.fna.fbcdn.net/safe_im...,836,92,16,36,11,4,True,left_all_posts,0
1,36123,2018-11-05T14:01:00+0000,219188754789679_2341230762585457,orlandeli,por orlandeli .,1,added_photos,https://scontent.fplu2-1.fna.fbcdn.net/v/t1.64...,13,1,0,0,0,3,True,left_all_posts,0
2,2233,2019-06-02T00:30:19+0000,371913239843447_864924640542302,audio julgamento bolsonaro tribunal militar jo...,audio do julgamento de bolsonaro no tribunal m...,437,added_video,https://scontent.fplu2-1.fna.fbcdn.net/v/t15.5...,201,18,21,1,1,1,True,left_all_posts,0
3,24570,2020-05-20T02:37:30+0000,292074710916413_1524289557744683,conheca plataforma enfrentamento pandemia coro...,conheca a plataforma para o enfrentamento da p...,617,added_video,https://scontent.fplu2-1.fna.fbcdn.net/v/t15.5...,591,7,7,289,19,23,True,left_all_posts,0
4,65984,2020-02-18T21:54:34+0000,127835925882_10157935816100883,forcas opostas fundo manutencao desenvolviment...,forcas opostas : o fundo de manutencao e desen...,85,shared_story,https://external.fplu2-1.fna.fbcdn.net/safe_im...,41,0,1,14,23,2,True,left_all_posts,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,3318,2020-07-15T20:01:40+0000,890711084350263_3241472082607473,bolsonaro ainda covid segundo novo exame,bolsonaro ainda esta com covid-19 segundo novo...,5,shared_story,https://external.fplu2-1.fna.fbcdn.net/safe_im...,277,3,0,38,0,3,True,right_all_posts,1
61057,10,2018-03-05T21:23:11+0000,890711084350263_1688888311199199,amanha julgamento hc lula ladrao stj transmiti...,e amanha o julgamento do hc do lula ladrao . s...,69,shared_story,https://external.fplu2-1.fna.fbcdn.net/safe_im...,296,6,5,0,17,10,True,right_all_posts,1
61058,33469,2019-05-07T21:14:44+0000,1965770023473808_2188151608163126,bolsonaro garante recursos colegios militares,bolsonaro garante recursos para colegios milit...,70,added_video,https://scontent.fplu2-1.fna.fbcdn.net/v/t15.1...,404,3,2,0,0,68,True,right_all_posts,1
61059,8196,2019-02-20T02:26:23+0000,550142935316937_847946782203216,presidente jair bolsonaro realidade petistas c...,nosso presidente jair bolsonaro . realidade qu...,214,added_photos,https://scontent.fplu2-1.fna.fbcdn.net/v/t1.64...,440,0,3,0,0,45,True,right_all_posts,1


In [6]:
text = posts_30K_each.pre_processed_message.values
labels = posts_30K_each.label.values

In [7]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [9]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [10]:
def b_tp(preds, labels):
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [11]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    model.train()
    
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        train_output.loss.backward()
        optimizer.step()
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

Epoch:  50%|█████     | 1/2 [06:30<06:30, 390.76s/it]


	 - Train loss: 0.5350
	 - Validation Accuracy: 0.7869
	 - Validation Precision: 0.8331
	 - Validation Recall: 0.7176
	 - Validation Specificity: 0.8561



Epoch: 100%|██████████| 2/2 [13:00<00:00, 390.25s/it]


	 - Train loss: 0.6182
	 - Validation Accuracy: 0.5022
	 - Validation Precision: 0.5022
	 - Validation Recall: 1.0000
	 - Validation Specificity: 0.0000



In [13]:
new_sentence = 'bozo'

test_ids = []
test_attention_mask = []

encoding = preprocessing(new_sentence, tokenizer)

test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Direita' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Esquerda'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  bozo
Predicted Class:  Direita


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [14]:
model.save_pretrained('/content/drive/MyDrive/TCC II/30K-2-epochs-bert-base-uncased/model')

In [15]:
train_idx
train = ",".join(map(str, train_idx));
train

'20492,51981,20369,2819,7488,56283,46461,55647,25092,41396,25303,53302,27588,41122,7369,33856,33455,51115,1943,48761,33421,53505,48410,2752,25058,54612,51117,38177,475,6568,53114,23296,2492,45508,47374,38277,29508,56097,26986,1157,9313,16329,22221,10782,43003,48243,42958,54400,59608,54909,10971,60625,38740,34238,13636,49502,21797,50932,54162,3977,13822,49995,47772,12756,24519,36508,26652,58819,19915,24958,4592,23774,41770,19586,5835,3183,46479,57326,51730,33168,59083,14456,2857,26808,2505,29092,41526,50116,50257,36699,34319,14801,19830,49412,54111,60115,11946,22514,39482,34078,29515,34278,10980,18417,21258,49078,28517,17534,10471,8371,45881,13991,40509,30529,36888,21479,20756,54531,13905,14134,40085,52539,2312,13087,2371,32210,44140,36285,7461,59844,35501,39002,60252,6887,4936,41742,41076,31119,36599,16440,18160,11599,26503,9055,60889,52132,34573,39445,50495,16253,42188,16352,4174,29940,42550,55206,42788,17611,39557,11143,50767,57522,58950,60113,6244,44200,44505,40078,45747,19144,42080

In [16]:
val_idx
val = ",".join(map(str, val_idx));
val

'46570,47559,36292,30643,20874,58510,53001,39920,15885,59362,43242,28561,8516,46587,27436,9043,37737,50175,4183,12966,35200,31980,55983,49448,11647,16210,29473,53872,51078,15737,46415,22160,29429,6240,55004,38879,30792,55256,37321,55980,11389,2715,1680,44612,55062,10775,50256,28191,7774,57158,22850,1464,45528,41228,205,13132,53224,59583,46267,43945,30759,49691,38244,1959,31172,48839,11186,58020,59520,60334,56847,59389,838,19375,18844,46989,54961,14943,32203,48311,33587,41853,4006,29970,13964,36852,54518,17988,28697,51161,108,30832,20989,58138,10598,31225,22839,7773,38929,28360,51849,14682,50146,53406,7333,30704,21854,31341,49435,276,53272,32372,58428,2808,18592,29164,58966,14665,11651,24808,22281,30017,15696,16797,37486,7881,14696,18640,5443,53725,26588,4899,5098,14130,11477,16668,24218,53577,14050,9657,31096,48788,21581,4363,48037,29936,58609,39600,17407,11579,54927,12503,9739,58835,14784,51398,11595,43910,21475,7651,38061,1817,32204,4366,49731,48380,54939,21833,30228,4746,60751,6388,